In [61]:
from importlib import reload
from tqdm import tqdm
import main
reload(main)
from main import *

In [53]:
panel_properties_file = "../reflectance_panel_example_data.json"
geopackage_file = "../data/20240529_uav_multispectral_orthos_20m/20240529_tarps_locations.gpkg"
orthophotos_dir = "../data/20240529_uav_multispectral_orthos_20m/orthophotos"

In [54]:
panels = load_panel_properties(panel_properties_file)
panel_locations = load_panel_locations(geopackage_file)
orthophoto_paths = get_orthophoto_paths(orthophotos_dir)
with rasterio.open(orthophoto_paths[0]) as orthophoto:
    number_of_bands = len(orthophoto.read())
    
assert len(panels) == len(panel_locations)
assert number_of_bands == len(panels[0]['bands'])

# 1. For each image extract each panel intensities for each band

In [55]:
intensities = np.zeros((len(orthophoto_paths), len(panel_locations), number_of_bands))
intensities.shape

(296, 3, 6)

In [56]:
orthophoto_paths = list(sorted(orthophoto_paths))

In [57]:
# Extract intensities
for photo_index, orthophoto_path in enumerate(tqdm(orthophoto_paths)):
    with rasterio.open(orthophoto_path) as orthophoto:
        photo = orthophoto.read()
        for panel_index, panel in enumerate(panel_locations):
            if not is_panel_in_orthophoto(orthophoto_path, panel):
                intensities[photo_index][panel_index] = np.full(len(photo), np.NaN)
                continue
            panel_intensities_per_band = extract(orthophoto, panel)
            for band_index, band in enumerate(photo):
                intensities[photo_index][panel_index][band_index] = panel_intensities_per_band[band_index]
intensities

100%|██████████| 296/296 [00:32<00:00,  9.07it/s]


array([[[nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan]],

       [[nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan]],

       [[nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan]],

       ...,

       [[nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan]],

       [[nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan]],

       [[nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan]]])

# Interpolate the intensities by panel and band

In [58]:
for panel_index, panel in enumerate(panel_locations):
    for band_index, band in enumerate(photo):
        intensities[:, panel_index, band_index] = interpolate(intensities[:, panel_index, band_index])
intensities

array([[[nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan]],

       [[nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan]],

       [[nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan]],

       ...,

       [[nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan]],

       [[nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan]],

       [[nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan]]])

# Convert photos by photo and band

In [62]:
unconvertable_photos = []
converted_photos = []
for photo_index, orthophoto_path in enumerate(tqdm(orthophoto_paths)):
    converted_bands = []
    for band_index, band in enumerate(photo):
        intensities_of_panels = intensities[photo_index, :, band_index]
        if np.isnan(intensities_of_panels).any():
            unconvertable_photos.append((photo_index, band_index))
            break
        coeffs = fit(intensities_of_panels, get_band_reflectance(panels, band_index))
        converted_bands.append(convert(band, coeffs))
    else:
        # for loop did not break, all bands were converted
        converted_photos.append(combine(converted_bands))

100%|██████████| 296/296 [00:09<00:00, 31.17it/s]


In [63]:
len(unconvertable_photos)

199

In [64]:
len(converted_photos)

97